In [4]:
import requests

url = "http://127.0.0.1:5000/remove_duplicates?day_range=1"

data = {
    "Message": ["Привіт", "Як справи?", "Привіт"],
    "MessageDate": ["2029-01-28 19:28:10.123", "2029-01-29 10:00:00.000", "2029-01-28 19:28:10.123"],
    "TelegramPostInfoID": [1, 2, 3]
}

response = requests.post(url, json=data)
print(response.json())



{'unique_ids': [1, 2]}


In [33]:
import requests
import sqlite3
url = "http://127.0.0.1:5001/api/fetch_posts"  
payload = {
    "channel": "news_channel",
    "start_date": "2024-02-01",
    "end_date": "2024-02-05",
    "model": "Bert"  
}

headers = {"Content-Type": "application/json"}

response = requests.post(url, json=payload, headers=headers)

print(response.status_code)
print(response.json())  


200
{'message': 'Повідомлення успішно отримані та збережені'}


In [4]:
DB_PATH = "/Users/ulanagusar/Desktop/ML_week/ML_week_nlp_solving/backend/database/database.db"
print(DB_PATH)
import sqlite3
def get_db_connection():

    conn = sqlite3.connect(DB_PATH)  
    conn.row_factory = sqlite3.Row 
    return conn


def fetch_table_contents(table_name):

    try:
        conn = get_db_connection()
        cursor = conn.cursor()


        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        conn.close()

        if rows:

            for row in rows:
                print(dict(row)) 
        else:
            print(f" {table_name} empty.")
    
    except Exception as e:
        print(f"{e}")
def show_all_tables():
    fetch_table_contents("TelegramPostInfo")



/Users/ulanagusar/Desktop/ML_week/ML_week_nlp_solving/backend/database/database.db


In [39]:
show_all_tables()


📌 Вміст таблиці TelegramPostInfo:
{'MessageID': 8420, 'Message': 'В парке винтокрылых машин 2-го Архангельского объединенного авиаотряда пополнение. Совершив прямой перелёт из Казани за пять часов в Васьково прибыл новый Ми-8МТВ-1. Данный вертолет планируют использовать для перевозки пассажиров труднодоступных территорий, обеспечения санитарной авиации и доставки грузов.\n\n#вертоновости', 'Channel': '@vertolatte', 'MessageDate': '2025-01-02 14:17:49.000', 'Name': '', 'Location': 'васьковый, казань', 'Weapons': 'тор', 'Observation': 'o', 'Discussion': 'd', 'Conclusion': 's', 'Recommendation': 'r'}


In [38]:
import requests
import sqlite3

API_URL = "http://127.0.0.1:5001/api/posts"  

def test_get_posts():

    try:
        response = requests.get(API_URL)  
        if response.status_code == 200:
            posts = response.json()

            for post in posts:
                print(post)
        else:
            print(f" {response.status_code}, response: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f" {e}")
test_get_posts()

{'Channel': '@vertolatte', 'Message': 'В парке винтокрылых машин 2-го Архангельского объединенного авиаотряда пополнение. Совершив прямой перелёт из Казани за пять часов в Васьково прибыл новый Ми-8МТВ-1. Данный вертолет планируют использовать для перевозки пассажиров труднодоступных территорий, обеспечения санитарной авиации и доставки грузов.\n\n#вертоновости', 'MessageDate': '2025-01-02 14:17:49.000', 'TelegramPostInfoID': 8420}


In [7]:
# experience.py
#!pip install pymorphy2 pymorphy2-dicts-ru
!pip install deeppavlov

import torch
import xgboost as xgb
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from pymorphy2 import MorphAnalyzer

def experience_bert(text, model_name="bodomerka/Milytary_exp_class_classification_sber_ai_based"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        return torch.argmax(outputs.logits).item()

def experience_xg_boost(text, xgb_model_path="xgb_model.ubj", bert_model_name="bodomerka/Milytary_exp_class_classification_sber_ai_based"):
    try:
        tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        transformer_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        transformer_model.to(device)
        
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            outputs = transformer_model(**inputs, output_hidden_states=True)
            embeddings = outputs.hidden_states[-1].mean(dim=1).cpu().numpy()
        
        xgb_model = xgb.Booster()
        xgb_model.load_model(xgb_model_path)
        dmatrix = xgb.DMatrix(embeddings)
        return int(xgb_model.predict(dmatrix)[0] > 0.5)
    except Exception as e:
        print(f"Error in experience_xg_boost: {e}")
        return 0

# ner.py
from deeppavlov import build_model, configs
from pymorphy2 import MorphAnalyzer

def get_ner_model():
    return build_model(configs.ner.ner_rus_bert, download=True)

def normalize_words(words):
    morph = MorphAnalyzer()
    return list({morph.parse(word)[0].normal_form for word in words})

def get_name(mes, ner_model):
    ner_results = ner_model([mes])
    names = {word for word, label in zip(ner_results[0][0], ner_results[1][0]) if label in ["B-PER", "I-PER"]}
    return normalize_words(names)

def get_location(mes, ner_model):
    ner_results = ner_model([mes])
    locations = {word for word, label in zip(ner_results[0][0], ner_results[1][0]) if label in ["B-LOC", "I-LOC"]}
    return normalize_words(locations)

def get_weapons(mes, weapons_list_path="weapon.txt"):
    try:
        with open(weapons_list_path, "r", encoding="utf-8") as file:
            weapons = {line.strip().lower() for line in file}
        return [w for w in weapons if w in mes.lower()]
    except FileNotFoundError:
        print("Warning: weapons.txt not found. Returning empty list.")
        return []

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 8.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 8.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 10.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 7.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.6
    Uninstalling pydantic-2.10.6:
      Successfully uninstalled pydantic-2.10.6
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.2
    Uninstalling numpy-2.2.2:
      Successfully uninstalled numpy-2.2.2
  Attempting uninstall: filelock


In [9]:
!pip install numpy==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 7.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deeppavlov 1.7.0 requires numpy<1.24, but you have numpy 1.26.4 which is incompatible.


In [8]:
ner_model = get_ner_model()
get_location("У Дніпрі новий наступ", ner_model)

2025-02-08 01:43:07.349 INFO in 'deeppavlov.core.data.utils'['utils'] at line 97: Downloading from http://files.deeppavlov.ai/v1/ner/ner_rus_bert_torch_new.tar.gz to /Users/ulanagusar/.deeppavlov/models/ner_rus_bert_torch_new.tar.gz
100%|██████████| 1.44G/1.44G [03:10<00:00, 7.57MB/s]
2025-02-08 01:46:17.976 INFO in 'deeppavlov.core.data.utils'['utils'] at line 284: Extracting /Users/ulanagusar/.deeppavlov/models/ner_rus_bert_torch_new.tar.gz archive into /Users/ulanagusar/.deeppavlov/models/ner_rus_bert_torch
/Users/ulanagusar/Desktop/ML_week/.conda/lib/python3.10/site-packages/sklearn/utils/__init__.py:108: FutureWarning: IS_PYPY is deprecated and will be removed in 1.7.
  


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [12]:
import spacy
from pymorphy2 import MorphAnalyzer

# Load spaCy model for Russian NER
nlp = spacy.load("ru_core_news_md")

def normalize_words(words):
    """Normalize words using pymorphy2 (lemmatization)."""
    morph = MorphAnalyzer()
    return list({morph.parse(word)[0].normal_form for word in words})

def get_name(mes):
    """Extracts person names (PER) from the text using spaCy."""
    doc = nlp(mes)
    names = {ent.text for ent in doc.ents if ent.label_ == "PER"}
    return normalize_words(names)

def get_location(mes):
    """Extracts locations (LOC) from the text using spaCy."""
    doc = nlp(mes)
    locations = {ent.text for ent in doc.ents if ent.label_ == "LOC"}
    return normalize_words(locations)

def get_weapons(mes, weapons_list_path="weapon.txt"):
    """Checks for weapon names in the text using a predefined list from 'weapon.txt'."""
    try:
        with open(weapons_list_path, "r", encoding="utf-8") as file:
            weapons = {line.strip().lower() for line in file}
        return [w for w in weapons if w in mes.lower()]
    except FileNotFoundError:
        print("Warning: weapons.txt not found. Returning an empty list.")
        return []

In [11]:
!python -m spacy download ru_core_news_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 9.2 MB/s eta 0:00:0000:0100:01m
  Using cached pymorphy3-2.0.2-py3-none-any.whl.metadata (1.8 kB)
  Using cached pymorphy3_dicts_ru-2.4.417150.4580142-py2.py3-none-any.whl.metadata (2.0 kB)
Using cached pymorphy3-2.0.2-py3-none-any.whl (53 kB)
Using cached pymorphy3_dicts_ru-2.4.417150.4580142-py2.py3-none-any.whl (8.4 MB)
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [14]:
get_location("В городе Милитополе идут бои")

['милитополь']

In [23]:
def experience_bert(text, model_name="bodomerka/Milytary_exp_class_classification_sber_ai_based"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        return torch.argmax(outputs.logits).item()

def experience_xg_boost(text, xgb_model_path="/Users/ulanagusar/Desktop/ML_week/ML_week_nlp_solving/api/xgb_model.ubj", bert_model_name="bodomerka/Milytary_exp_class_classification_sber_ai_based"):
    try:
        tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        transformer_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        transformer_model.to(device)
        
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            outputs = transformer_model(**inputs, output_hidden_states=True)
            embeddings = outputs.hidden_states[-1].mean(dim=1).cpu().numpy()
        
        xgb_model = xgb.Booster()
        xgb_model.load_model(xgb_model_path)
        dmatrix = xgb.DMatrix(embeddings)
        return int(xgb_model.predict(dmatrix)[0] > 0.5)
    except Exception as e:
        print(f"Error in experience_xg_boost: {e}")
        return 0

In [18]:
experience_bert("завтра будет прорив , берите гранати")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0

In [24]:
experience_xg_boost("завтра будет прорив , берите гранати")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Error in experience_xg_boost: [02:12:42] /Users/runner/work/xgboost/xgboost/src/common/json.cc:390: Expecting: "L", got: "34 ", around character position: 2
    {"learner"
    ~^~~~~~~~~
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001453a8428 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x000000014549e180 xgboost::JsonReader::Error(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>) const + 808
  [bt] (2) 3   libxgboost.dylib                    0x000000014549e714 xgboost::JsonReader::Expect(signed char, signed char) + 392
  [bt] (3) 4   libxgboost.dylib                    0x00000001454a0124 xgboost::UBJReader::DecodeStr() + 184
  [bt] (4) 5   libxgboost.dylib                    0x0000000145498e3c xgboost::UBJReader::ParseObject() + 144
  [bt] (5) 6   libxgboost.dylib                    0x000000014549fad0 xgboost::UBJReader::Parse() + 708
  [bt] (6) 7   libxgboost.dylib    

0

In [26]:
experience_xg_boost("у нас новие гранати ПМ16")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Error in experience_xg_boost: [02:13:57] /Users/runner/work/xgboost/xgboost/src/common/json.cc:390: Expecting: "L", got: "34 ", around character position: 2
    {"learner"
    ~^~~~~~~~~
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001453a8428 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x000000014549e180 xgboost::JsonReader::Error(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>) const + 808
  [bt] (2) 3   libxgboost.dylib                    0x000000014549e714 xgboost::JsonReader::Expect(signed char, signed char) + 392
  [bt] (3) 4   libxgboost.dylib                    0x00000001454a0124 xgboost::UBJReader::DecodeStr() + 184
  [bt] (4) 5   libxgboost.dylib                    0x0000000145498e3c xgboost::UBJReader::ParseObject() + 144
  [bt] (5) 6   libxgboost.dylib                    0x000000014549fad0 xgboost::UBJReader::Parse() + 708
  [bt] (6) 7   libxgboost.dylib    

0

In [29]:
!pip install transformers sentencepiece
!pip install transformers torch


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5Tokenizer, T5ForConditionalGeneration, MBartTokenizer, MBartForConditionalGeneration

def gen_o(text: str, model_name: str = "cointegrated/rut5-base-absum"):
    """Генерация спостереження (Observation)"""
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    prompt = "У чем проблема: " + text
    input_ids = tokenizer.encode(prompt, return_tensors="pt", truncation=True, max_length=512)
    output_ids = model.generate(input_ids, num_beams=5, early_stopping=True, max_length=256)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

def gen_d(text, max_length=512, min_length=50, lang_code="ru_RU"):
    """Генерация обоснования (Discussion)"""
    model_name = "facebook/mbart-large-50"
    tokenizer = MBartTokenizer.from_pretrained(model_name)
    model = MBartForConditionalGeneration.from_pretrained(model_name)

    # Переконаємося, що lang_code існує
    if lang_code not in tokenizer.lang_code_to_id:
        print(f"Warning: lang_code {lang_code} not found! Using 'ru'.")
        lang_code = "ru"

    prompt = "Объясните, почему это произошло: " + text  # Повернув промпт!
    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True)
    output_ids = model.generate(
        inputs, max_length=max_length, min_length=min_length, length_penalty=2.0,
        num_beams=4, early_stopping=True, forced_bos_token_id=tokenizer.lang_code_to_id[lang_code]
    )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

def gen_c(text: str, model_name: str = "IlyaGusev/rut5_base_sum_gazeta"):
    """Генерация вывода (Conclusion)"""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    prompt = "Сформулируйте краткий и четкий вывод на основе текста: " + text
    input_ids = tokenizer.encode(prompt, return_tensors="pt", truncation=True, max_length=512)
    output_ids = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

def gen_r(text: str, model_name: str = "cointegrated/rut5-base-absum"):
    """Генерация рекомендации (Recommendation)"""
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    prompt = "Опишите, что произошло и что нужно делать: " + text
    input_ids = tokenizer.encode(prompt, return_tensors="pt", truncation=True, max_length=512)
    output_ids = model.generate(input_ids, num_beams=5, early_stopping=True, max_length=256)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.5 MB/s eta 0:00:00a 0:00:01


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
gen_r("войска наступают")

ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [36]:
from pyrogram import Client
import asyncio

def fetch_messages(start_date, end_date, channel_name):
    if channel_name == "Вертолатте":
        channel = "@vertolatte"
    elif channel_name == "ДРОННИЦА":
        channel = "@dronnitsa"
    else:
        channel = "@donbassrussiazvo"
    
    messages, dates, channels, ids = [], [], [], []
    
    with Client("military_bot", API_ID, API_HASH) as app:
        try:
            loop = asyncio.get_event_loop()
            chat = loop.run_until_complete(app.get_chat(channel))
            
            for message in loop.run_until_complete(app.get_chat_history(chat.id)):
                if not message.date or message.date < start_date:
                    break
                
                if start_date <= message.date <= end_date:
                    message_text = message.text if message.text else message.caption
                    if message_text:
                        messages.append(message_text)
                        print(message_text)
                        dates.append(message.date.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3])
                        channels.append(channel)
                        ids.append(message.id)
        except Exception as e:
            print(f"Error receiving messages: {e}")
    
    return messages, dates, channels, ids

messages, dates, channels, ids = fetch_messages(start_date, end_date, "ДРОННИЦА")

Error receiving messages: 'coroutine' object has no attribute 'get_chat'


/var/folders/nq/srzxpdh14hlgl_4b4d260fm40000gn/T/ipykernel_51176/1212648523.py:36: RuntimeWarning: coroutine 'Start.start' was never awaited
  messages, dates, channels, ids = fetch_messages(start_date, end_date, "ДРОННИЦА")


In [37]:
import requests

# URL вашого Flask сервера (локально на порті 5003)
API_URL = "http://127.0.0.1:5003/predict/bert"

# Тестовий текст для класифікації
test_data = {"text": "Це тестовий текст для моделі BERT"}

# Відправлення POST-запиту
response = requests.post(API_URL, json=test_data)

# Виведення результату
print(response.status_code)
print(response.json())


200
{'prediction': 0}


In [ ]:
# # Load spaCy model for Russian NER
# nlp = spacy.load("ru_core_news_md")

# def normalize_words(words):
#     """Normalize words using pymorphy2 (lemmatization)."""
#     morph = MorphAnalyzer()
#     return list({morph.parse(word)[0].normal_form for word in words})

# def get_name(mes):
#     """Extracts person names (PER) from the text using spaCy."""
#     doc = nlp(mes)
#     names = {ent.text for ent in doc.ents if ent.label_ == "PER"}
#     return normalize_words(names)

# def get_location(mes):
#     """Extracts locations (LOC) from the text using spaCy."""
#     doc = nlp(mes)
#     locations = {ent.text for ent in doc.ents if ent.label_ == "LOC"}
#     return normalize_words(locations)

# def get_weapons(mes, weapons_list_path="weapon.txt"):
#     """Checks for weapon names in the text using a predefined list from 'weapon.txt'."""
#     try:
#         with open(weapons_list_path, "r", encoding="utf-8") as file:
#             weapons = {line.strip().lower() for line in file}
#         return [w for w in weapons if w in mes.lower()]
#     except FileNotFoundError:
#         print("Warning: weapons.txt not found. Returning an empty list.")
#         return []
